<a href="https://colab.research.google.com/github/wojtwkg/lecture25.03/blob/main/WG_rekomendacja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy==1.23.5

In [2]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2461564 sha256=dccc4d2da84d1e09f30446375697b0f32732dcd8b4990c545e38af764e2314a9
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [5]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate, GridSearchCV
from surprise.prediction_algorithms.matrix_factorization import SVD
from surprise.prediction_algorithms.knns import KNNBasic, KNNBaseline

In [7]:
filmy = pd.read_csv('movies.csv')

oceny = pd.read_csv('ratings.csv')

In [10]:
min_ocena = oceny.rating.min()

max_ocena = oceny.rating.max()


reader = Reader(rating_scale=(min_ocena, max_ocena))

data = Dataset.load_from_df(oceny[['userId', 'movieId', 'rating']], reader)

In [12]:
algorytm = SVD()

cross_validate(algorytm, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8768  0.8765  0.8663  0.8738  0.8780  0.8743  0.0042  
MAE (testset)     0.6738  0.6754  0.6620  0.6710  0.6752  0.6715  0.0050  
Fit time          2.61    1.73    1.57    2.13    2.52    2.11    0.41    
Test time         0.49    0.15    0.28    0.22    0.14    0.26    0.13    


{'test_rmse': array([0.87678862, 0.87646347, 0.86634759, 0.87381926, 0.87801307]),
 'test_mae': array([0.67381456, 0.6754346 , 0.66198551, 0.6709851 , 0.67516328]),
 'fit_time': (2.6075093746185303,
  1.7292673587799072,
  1.5667262077331543,
  2.1314797401428223,
  2.524221181869507),
 'test_time': (0.49251532554626465,
  0.14751172065734863,
  0.2786068916320801,
  0.21940994262695312,
  0.1393437385559082)}

In [14]:
zbior_treningowy = data.build_full_trainset()

algorytm.fit(zbior_treningowy)

In [18]:
wszystkie_filmy = set(filmy['movieId'])

ocenione_filmy = set(oceny[oceny['userId'] == 600]['movieId'])

nieocenione_filmy = wszystkie_filmy - ocenione_filmy

predykcje = [ (movie_id, algorytm.predict(600, movie_id).est) for movie_id in nieocenione_filmy]

predykcje.sort(key=lambda x: x[1], reverse=True)

top_10 = predykcje[:10]

polecane_filmy = pd.DataFrame(top_10, columns=['movieId', 'predicted_rating'])

polecane_filmy = polecane_filmy.merge(filmy, on='movieId')

print(polecane_filmy[['movieId', 'title', 'predicted_rating']])

   movieId                                title  predicted_rating
0    56782           There Will Be Blood (2007)          4.350637
1     5995                  Pianist, The (2002)          4.217822
2     3152        Last Picture Show, The (1971)          4.217097
3     6016  City of God (Cidade de Deus) (2002)          4.198546
4     1203                  12 Angry Men (1957)          4.121996
5     3451  Guess Who's Coming to Dinner (1967)          4.083297
6    48516                 Departed, The (2006)          4.080114
7     1228                   Raging Bull (1980)          4.036434
8     1221       Godfather: Part II, The (1974)          4.027753
9       16                        Casino (1995)          4.019611
